In [1]:
%pip install -U datasets sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; pla

In [2]:
import torch
import gc
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sacrebleu.metrics import BLEU
from tqdm.auto import tqdm
import re
from datasets import load_dataset

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def clean_text(text):
    """Chuẩn hóa văn bản bằng cách chuyển sang chữ thường và loại bỏ các ký tự không cần thiết."""
    text = re.sub(r'\s+([,.!?;:"])', r'\1', text)
    text = re.sub(r'\s+', ' ', text)
    return text.lower().strip()

dataset = load_dataset("Angelectronic/IWSLT15_English_Vietnamese")['train']['translation']
en = [clean_text(sample['en']) for sample in dataset]
vi = [clean_text(sample['vi']) for sample in dataset]

val_size = 10000
val_en, val_vi = en[-val_size:], vi[-val_size:]

README.md:   0%|          | 0.00/538 [00:00<?, ?B/s]

(…)-00000-of-00001-32a2d2de76910062.parquet:   0%|          | 0.00/18.4M [00:00<?, ?B/s]

(…)-00000-of-00001-90850d7d3fd03986.parquet:   0%|          | 0.00/188k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/133166 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1268 [00:00<?, ? examples/s]

In [5]:
models_to_test = [
    "vinai/vinai-translate-en2vi-v2",
    "Helsinki-NLP/opus-mt-en-vi"
]

for model_name in models_to_test:
    print("\n" + "="*60)
    print(f"TESTING MODEL: {model_name}")
    print("="*60)

    try:
        print("loading model and tokenizer...")
        if "vinai" in model_name:
            tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang="en_XX")
        else:
            tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(DEVICE)
        model.eval()

        hypotheses = []
        
        with torch.no_grad():
            for sentence in tqdm(val_en, desc=f"Using {model_name.split('/')[-1]}"):
                inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128).to(DEVICE)
                translated_tokens = model.generate(
                    **inputs,
                    max_length=100,
                    num_beams=5,
                    early_stopping=True,
                    no_repeat_ngram_size=2
                )
                translation = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
                hypotheses.append(translation)

        bleu = BLEU()
        score = bleu.corpus_score(hypotheses, [val_vi])

        print(f"\n--- RESULT FOR {model_name} ---")
        print(f"   => Bleu: {score.score:.2f}")
        
        print(val_en[:3])
        print(hypotheses[:3])
        print(val_vi[:3])

    except Exception as e:
        print(f"idk whats going on")

    finally:
        print("\nsetting freedom for GPU and RAM...")
        del model
        del tokenizer
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

print("\ndone ig.")


TESTING MODEL: vinai/vinai-translate-en2vi-v2
loading model and tokenizer...


config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.41M [00:00<?, ?B/s]

2025-06-29 19:09:55.302575: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751224195.478484      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751224195.529648      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Using opus-mt-en-vi:   0%|          | 0/10000 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]


--- RESULT FOR Helsinki-NLP/opus-mt-en-vi ---
   => Bleu: 23.45
["also in this vicinity is a region that 's selectively involved in processing visual motion, like these moving dots here, and that 's in yellow at the bottom of the brain, and near that is a region that responds when you look at images of bodies and body parts like these, and that region is shown in lime green at the bottom of the brain.", "now all these regions i 've shown you so far are involved in specific aspects of visual perception.", 'do we also have specialized brain regions for other senses, like hearing?']
['trong khu vực này là một vùng được chọn lọc tham gia trong việc xử lý chuyển động hình ảnh, như những chấm di chuyển ở đây, và đó là màu vàng ở phía dưới của não, gần đó có một khu đất phản ứng khi bạn nhìn vào hình tượng của cơ thể và các bộ phận cơ thể như thế này, vùng đó được hiển thị trong vôi màu xanh ở dưới cùng của bộ não.', 'Bây giờ tất cả các vùng mà tôi đã chỉ cho bạn đến nay đều liên quan đến c